In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np

In [45]:
sing_df = pd.read_excel("sheets/land_use_constrained_data.xlsx", sheet_name=1, header=5, usecols="B:V", nrows=9)

sf_df = pd.read_excel("sheets/land_use_constrained_data.xlsx", sheet_name=1, header=55, usecols="B:V", nrows=11)

sing_df

,Planning zone,# of Zone Types,% Open Space,% Residential,% Commercial,% Industrial,% Mixed Use,% Paved Area,Distance covered in 30 minutes using Public Transportation (km),Distance covered in 30 minutes using a Car (km),...,Distance covered in 30 minutes Cycling (km),"Cost of daily commute per capita, per km using bus (S$)",Cost of daily commute per capita per km using MRT (S$),Cost of daily commute per capita per km Driving (S$),% households with income >= USD8000/month,% residents with high school degrees,% residents with university degrees,% residents employed,% residents 15y + literate,% residents 15y + literate in 2+ languages
0,Bukit Merah,5,0.387,0.283,0.121,0.021,0.063,0.126,6.13,14.66,...,6.84,0.13,0.06,0.30,0.4105,0.6912,0.3211,0.6121,0.9546,0.6615
1,Bukit Timah,5,0.030,0.655,0.004,0.003,0.156,0.152,1.02,15.41,...,4.94,0.13,0.06,0.30,0.6971,0.9217,0.6633,0.5877,0.9912,0.7409
2,Geylang,5,0.063,0.416,0.233,0.023,0.073,0.193,4.16,22.02,...,6.11,0.13,0.06,0.29,0.4081,0.6993,0.3013,0.6311,0.9540,0.6697
3,Kallang,5,0.285,0.288,0.126,0.027,0.063,0.211,5.96,19.98,...,6.17,0.13,0.06,0.30,0.4116,0.6888,0.3311,0.6217,0.9513,0.6481
4,Novena,5,0.118,0.579,0.054,0.022,0.059,0.168,10.74,21.14,...,6.41,0.13,0.06,0.30,0.5478,0.8130,0.4786,0.6261,0.9740,0.7019
5,Queenstown,5,0.115,0.454,0.156,0.045,0.072,0.159,11.36,12.65,...,7.32,0.12,0.06,0.29,0.4482,0.7317,0.3735,0.6015,0.9627,0.6721
6,Tanglin,5,0.191,0.621,0.030,0.016,0.030,0.112,4.63,18.58,...,6.34,0.15,0.08,0.32,0.7249,0.9507,0.7377,0.6097,0.9949,0.7162
7,Toa Payoh,5,0.077,0.457,0.153,0.001,0.109,0.203,5.62,21.01,...,6.71,0.12,0.06,0.29,0.4084,0.7269,0.3196,0.6065,0.9584,0.6700
8,CBD Area,5,0.319,0.130,0.190,0.040,0.126,0.193,6.23,18.32,...,5.23,0.16,0.08,0.32,0.5858,0.8288,0.5620,0.6507,0.9742,0.7050


In [60]:
sing_df.iloc[:, 8:15]

,Distance covered in 30 minutes using Public Transportation (km),Distance covered in 30 minutes using a Car (km),Distance covered in 30 minutes Walking (km),Distance covered in 30 minutes Cycling (km),"Cost of daily commute per capita, per km using bus (S$)",Cost of daily commute per capita per km using MRT (S$),Cost of daily commute per capita per km Driving (S$)
0,6.13,14.66,2.18,6.84,0.13,0.06,0.30
1,1.02,15.41,2.04,4.94,0.13,0.06,0.30
2,4.16,22.02,1.95,6.11,0.13,0.06,0.29
3,5.96,19.98,2.02,6.17,0.13,0.06,0.30
4,10.74,21.14,2.16,6.41,0.13,0.06,0.30
5,11.36,12.65,2.17,7.32,0.12,0.06,0.29
6,4.63,18.58,2.17,6.34,0.15,0.08,0.32
7,5.62,21.01,1.80,6.71,0.12,0.06,0.29
8,6.23,18.32,2.19,5.23,0.16,0.08,0.32


In [97]:
def land_use_metric(arr):
    """arr should be the land use metrics that are not paved area"""
    # goal: even split between the 5 land use types, reserving 10% of area for paved areas 
    goal_ratio = 0.9/4
    ratios = [i - goal_ratio for i in arr]
    return np.sum(ratios)

def make_metrics(df, city):
    df["Sum Land Use"] = df.iloc[:, 2:8].sum(1)
    adjusted_land_use_df = df.iloc[:, 2:8].apply(lambda x: x/df["Sum Land Use"])
    diversity_metric = pd.Series(adjusted_land_use_df.iloc[:,0:4].apply(lambda x: land_use_metric(x), axis=1), name="Land Use Diversity")
    # print(adjusted_land_use_df, diversity_metric)
    transport_metric = df.iloc[:, 8]
    qol_metric = df.iloc[:, -6]

    df2 = pd.DataFrame([diversity_metric, transport_metric, qol_metric, pd.Series([city]*len(qol_metric))], index=["land", "transport", "education", "city"]).T

    return df2

In [100]:
sing_dfa = make_metrics(sing_df, "singapore")
sf_dfa = make_metrics(sing_df, "san francisco")
sf_dfa

,land,transport,education,city
0,-0.088811,6.13,0.6912,san francisco
1,-0.208,1.02,0.9217,san francisco
2,-0.165734,4.16,0.6993,san francisco
3,-0.174,5.96,0.6888,san francisco
4,-0.127,10.74,0.813,san francisco
5,-0.130769,11.36,0.7317,san francisco
6,-0.042,4.63,0.9507,san francisco
7,-0.212,5.62,0.7269,san francisco
8,-0.219639,6.23,0.8288,san francisco


In [96]:
make_metrics(sing_df)

analyze_df = pd.DataFrame([diversity_metric, transport_metric, qol_metric, pd.Series(["singapore"]*len(qol_metric))], index=["land", "transport", "education", "city"]).T

(0   -0.088811
 1   -0.208000
 2   -0.165734
 3   -0.174000
 4   -0.127000
 5   -0.130769
 6   -0.042000
 7   -0.212000
 8   -0.219639
 Name: Land Use Diversity, dtype: float64,
 0     6.13
 1     1.02
 2     4.16
 3     5.96
 4    10.74
 5    11.36
 6     4.63
 7     5.62
 8     6.23
 Name: Distance covered in 30 minutes using Public Transportation (km), dtype: float64,
 0    0.6912
 1    0.9217
 2    0.6993
 3    0.6888
 4    0.8130
 5    0.7317
 6    0.9507
 7    0.7269
 8    0.8288
 Name: % residents with high school degrees, dtype: float64)

In [46]:
sing_df["Sum Land Use"] = sing_df.iloc[:, 2:8].sum(1)
adjusted_land_use_df = sing_df.iloc[:, 2:8].apply(lambda x: x/sing_df["Sum Land Use"])
diversity_metric = pd.Series(adjusted_land_use_df.iloc[:,0:4].apply(lambda x: land_use_metric(x), axis=1), name="Land Use Diversity")
print(adjusted_land_use_df, diversity_metric)

,% Open Space,% Residential,% Commercial,% Industrial,% Mixed Use,% Paved Area
0,0.386613,0.282717,0.120879,0.020979,0.062937,0.125874
1,0.030000,0.655000,0.004000,0.003000,0.156000,0.152000
2,0.062937,0.415584,0.232767,0.022977,0.072927,0.192807
3,0.285000,0.288000,0.126000,0.027000,0.063000,0.211000
4,0.118000,0.579000,0.054000,0.022000,0.059000,0.168000
5,0.114885,0.453546,0.155844,0.044955,0.071928,0.158841
6,0.191000,0.621000,0.030000,0.016000,0.030000,0.112000
7,0.077000,0.457000,0.153000,0.001000,0.109000,0.203000
8,0.319639,0.130261,0.190381,0.040080,0.126253,0.193387


In [64]:
transport_metric = sing_df.iloc[:, 8]
qol_metric = sing_df.iloc[:, -6]
qol_metric

0    0.6912
1    0.9217
2    0.6993
3    0.6888
4    0.8130
5    0.7317
6    0.9507
7    0.7269
8    0.8288
Name: % residents with high school degrees, dtype: float64

In [94]:
analyze_df = pd.DataFrame([diversity_metric, transport_metric, qol_metric, pd.Series(["singapore"]*len(qol_metric))], index=["land", "transport", "education", "city"]).T
analyze_df

,land,transport,education,city
0,-0.088811,6.13,0.6912,singapore
1,-0.208,1.02,0.9217,singapore
2,-0.165734,4.16,0.6993,singapore
3,-0.174,5.96,0.6888,singapore
4,-0.127,10.74,0.813,singapore
5,-0.130769,11.36,0.7317,singapore
6,-0.042,4.63,0.9507,singapore
7,-0.212,5.62,0.7269,singapore
8,-0.219639,6.23,0.8288,singapore


In [74]:
model_sing = ols("""education ~ C(land) + C(transport) +
               C(land):C(transport)""", data=analyze_df).fit()



In [ ]:
sm.stats.anova_lm(model_sing, typ=2)

In [88]:
sf_metrics = make_metrics(sf_df)
sf_metrics

(0    -0.162000
 1    -0.133766
 2    -0.208000
 3    -0.171000
 4    -0.195000
 5    -0.356000
 6    -0.140000
 7    -0.175000
 8    -0.225000
 9    -0.130230
 10   -0.215000
 Name: Land Use Diversity, dtype: float64,
 0     1.69
 1     2.98
 2     3.17
 3     2.88
 4     6.07
 5     7.57
 6     2.80
 7     4.26
 8     8.85
 9     4.43
 10    7.08
 Name: Distance covered in 30 minutes using Public Transportation (km), dtype: float64,
 0     0.9106
 1     0.9732
 2     0.7666
 3     0.8300
 4     0.8945
 5     0.8945
 6     0.9382
 7     0.9653
 8     0.8549
 9     0.7916
 10    0.7667
 Name: % residents with high school degrees, dtype: float64)

0    san francisco
1    san francisco
2    san francisco
3    san francisco
4    san francisco
5    san francisco
6    san francisco
7    san francisco
8    san francisco
dtype: object

In [95]:
sf_analysis_df = pd.DataFrame([diversity_metric, transport_metric, qol_metric, pd.Series(["san francisco"]*len(qol_metric))], index=["land", "transport", "education", "city"]).T

## Example

In [3]:
#create DataFrame
df = pd.DataFrame({'program': np.repeat([1, 2], 20),
                   'gender': np.tile(np.repeat(['M', 'F'], 10), 2),
                   'division': np.tile(np.repeat([1, 2], 5), 4),
                   'height': [7, 7, 8, 8, 7, 6, 6, 5, 6, 5,
                              5, 5, 4, 5, 4, 3, 3, 4, 3, 3,
                              6, 6, 5, 4, 5, 4, 5, 4, 4, 3,
                              2, 2, 1, 4, 4, 2, 1, 1, 2, 1]})

In [6]:
df.head()

,program,gender,division,height
0,1,M,1,7
1,1,M,1,7
2,1,M,1,8
3,1,M,1,8
4,1,M,1,7


In [5]:
model = ols("""height ~ C(program) + C(gender) + C(division) +
               C(program):C(gender) + C(program):C(division) + C(gender):C(division) +
               C(program):C(gender):C(division)""", data=df).fit()

sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(program),3.610000e+01,1.0,6.563636e+01,2.983934e-09
C(gender),6.760000e+01,1.0,1.229091e+02,1.714432e-12
C(division),1.960000e+01,1.0,3.563636e+01,1.185218e-06
C(program):C(gender),4.718528e-29,1.0,8.579142e-29,1.000000e+00
C(program):C(division),4.000000e-01,1.0,7.272727e-01,4.001069e-01
C(gender):C(division),1.000000e-01,1.0,1.818182e-01,6.726702e-01
C(program):C(gender):C(division),1.000000e-01,1.0,1.818182e-01,6.726702e-01
Residual,1.760000e+01,32.0,NaN,NaN


In [ ]:
# get data 
# do two way anova 